In [783]:
import requests
import base64
import re
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [185]:
client_id= "ceea23cb0884450b888029d74eba121f"
client_secret= "4b2da38633344c12bce0cfa687472e86"
redirect_uri = 'http://localhost:8888/callback'

In [186]:
client_credentials = f"{client_id}:{client_secret}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

In [187]:
# Request the acces token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)
if response.status_code == 200:
    access_token = response.json()['access_token']
    print(access_token)
else:
    print("Error")
    exit()

BQDzH5ifaaqThzVzaZXC71NdZUkeoR1Hr_fKcFD28CjIRRm8bEbBalPWwfqSPEQLqAgZBmCagh8uy1h3lBgjB83M1zNSkPmd7FOQyESR543NUvxADqY


In [188]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth
auth_manager = spotipy.oauth2.SpotifyOAuth(
    client_id='your_client_id',
    client_secret='your_client_secret',
    redirect_uri='your_redirect_uri',
    scope='user-library-read', 
)

In [497]:
def get_playlist_data(playlist_id,access_token):
    # setup spotify with the access token
    sp = spotipy.Spotify(auth=access_token)
    
    #Get the track from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')
    
    #Extract relevent information and store in a list of dictionaries
    data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']
        
        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None
        
        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None
            
        # Get popularity of the track
        try:
            track_info = sp.track(tack_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None
            
        # Add additional track information to the track data
        track_data = {
            'Track Name' : track_name,
            'Artists' : artists,
            'Album Name': album_name,
            'Album Id': album_id,
            'Track ID':track_id,
            'Popularity':popularity,
            'Release Date':release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit URLs': track_info.get('external_urls',{}).get('splotify',None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
        }
        
        data.append(track_data)
        
    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    
    return df

In [192]:
playlist_id1 = '37i9dQZF1DX0A8zVl7p82B'
christmas_pop = get_playlist_data(playlist_id1,access_token)
christmas_pop[:5]

,Track Name,Artists,Album Name,Album Id,Track ID,Popularity,Release Date,Duration (ms),Explicit URLs,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Santa Tell Me,Ariana Grande,Santa Tell Me,27MNgBEnLCKoafz1g2Zu97,0lizgQ7Qw35od7CYaoMBZb,None,2014-11-24,204093,None,0.525,0.621,7,-7.364,1,0.1160,0.04890,0.00000,0.2940,0.591,191.900
1,Underneath the Tree,Kelly Clarkson,Wrapped In Red,7Jahqd1kx9Qau0E9x9iZj6,3YZE5qDV7u1ZD1gZc47ZeR,None,2013-10-29,229640,None,0.509,0.812,8,-5.439,1,0.0537,0.00128,0.00002,0.2140,0.689,159.878
2,All I Want for Christmas Is You,Mariah Carey,Merry Christmas,61ulfFSmmxMhc2wCdmdMkN,0bYg9bo50gSsH3LtXe2SQn,None,1994-10-28,241107,None,0.336,0.627,7,-7.463,1,0.0384,0.16400,0.00000,0.0708,0.350,150.273
3,Rockin' Around The Christmas Tree,Brenda Lee,Merry Christmas From Brenda Lee,34wa3zf2prXFMk47t9zHFG,2EjXfH91m7f8HiJN1yQg97,None,1964-10-19,126267,None,0.589,0.472,8,-8.749,1,0.0502,0.61400,0.00000,0.5050,0.898,67.196
4,Last Christmas - Remastered,Wham!,Twenty Five,5HaOXSCK1IlwzV4ve7CUKY,6wn2nmFn3wDuiMldRiuRuL,None,2006-11-11,267160,None,0.757,0.618,11,-7.964,0,0.0282,0.26200,0.00000,0.2210,0.861,106.856


In [193]:
print(christmas_pop.isnull().sum())

Track Name           0
Artists              0
Album Name           0
Album Id             0
Track ID             0
Popularity          90
Release Date         0
Duration (ms)        0
Explicit URLs       90
Danceability         0
Energy               0
Key                  0
Loudness             0
Mode                 0
Speechiness          0
Acousticness         0
Instrumentalness     0
Liveness             0
Valence              0
Tempo                0
dtype: int64


In [195]:
playlist_id2 = '6UeSakyzhiEt4NB3UAd6NQ'
bilboard100 = get_playlist_data(playlist_id2,access_token)
playlist_id3 = '5TZkls9cEOzWDR6qCxwDot'
hiphop2023 = get_playlist_data(playlist_id3,access_token)
playlist_id4 = '37i9dQZF1DX6wfQutivYYr'
hotitalia = get_playlist_data(playlist_id4,access_token)
playlist_id5 = '37i9dQZF1DZ06evO4lAAFJ'
rosalia = get_playlist_data(playlist_id5,access_token)
playlist_id6 ='37i9dQZF1DZ06evO4kToQ3'
libianca = get_playlist_data(playlist_id6,access_token)
playlist_id7 = '37i9dQZF1DX2apWzyECwyZ'
badbunny = get_playlist_data(playlist_id7,access_token)
playlist_id8 ='37i9dQZF1DZ06evO0jO79m'
shakira = get_playlist_data(playlist_id8,access_token)
playlist_id9 = '37i9dQZF1DZ06evO2Xr2ut'
quevedo = get_playlist_data(playlist_id9,access_token)

In [196]:
playlist_id10 ='37i9dQZF1DX9tPFwDMOaN1'
kpop1 = get_playlist_data(playlist_id10,access_token)
playlist_id11 ='37i9dQZF1DXe5W6diBL5N4'
kpop2 = get_playlist_data(playlist_id11,access_token)
playlist_id12 = '37i9dQZF1DX14fiWYoe7Oh'
kpop3 = get_playlist_data(playlist_id12,access_token)

In [201]:
playlist_id13 ='37i9dQZEVXbNG2KDcFcKOF'
globalhit = get_playlist_data(playlist_id13,access_token)
playlist_id14 ='37i9dQZF1DXdbRLJPSmnyq'
jpophit = get_playlist_data(playlist_id14,access_token)
playlist_id15 = '6mtYuOxzl58vSGnEDtZ9uB'
pop1 = get_playlist_data(playlist_id15,access_token)
playlist_id16 ='3YW5W3NwxUHOWwu1Pk8zsu'
pop2 = get_playlist_data(playlist_id16,access_token)
playlist_id17 ='1TNg7JCxifAjwrnQARimex'
pop4 = get_playlist_data(playlist_id17,access_token)
playlist_id18 = '5kFqXhqpGxL2VbgRyF4CAW'
pop3 = get_playlist_data(playlist_id18,access_token)

In [605]:
data = pd.concat([christmas_pop,bilboard100,
                  hiphop2023,hotitalia,rosalia,libianca,badbunny,
                  shakira,quevedo, kpop1,kpop2,kpop3, globalhit, jpophit, pop1, pop2,pop3,pop4 ])
# Find duplicates without dropping them
duplicates = data[data.duplicated(keep=False)]
df.to_csv('data1.csv', index=False)
display(pd.DataFrame(duplicates).tail(100))

,Track Name,Artists,Album Name,Album Id,Track ID,Popularity,Release Date,Duration (ms),Explicit URLs,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
40,Daylight,David Kushner,Daylight,6NcI39WPu4kY6Tul11nhSv,1odExI7RdWc4BT515LTAwj,None,2023-04-14,212954,None,0.508,0.430,2,-9.475,0,0.0335,0.83000,0.000441,0.0930,0.324,130.090
41,I KNOW ?,Travis Scott,UTOPIA,18NOKLkZETa4sWwLMIm0UZ,6wsqVwoiVH2kde4k4KKAFU,None,2023-07-28,211582,None,0.927,0.619,5,-4.441,0,0.0539,0.01860,0.000000,0.1040,0.817,117.995
44,Barbie World (with Aqua) [From Barbie The Album],"Nicki Minaj, Ice Spice, Aqua",Barbie World (with Aqua) [From Barbie The Album],5CM66hwjlbZ06LhONWXOAs,741UUVE2kuITl0c6zuqqbO,None,2023-06-23,109750,None,0.770,0.580,0,-8.393,1,0.2470,0.51900,0.000127,0.2330,0.753,144.072
49,Standing Next to You,Jung Kook,GOLDEN,5pSk3c3wVwnb2arb6ohCPU,2KslE17cAJNHTsI2MI0jb2,None,2023-11-03,206020,None,0.711,0.809,2,-4.389,0,0.0955,0.04470,0.000000,0.3390,0.816,106.017
51,Strangers,Kenya Grace,Strangers,18ogtNq9F7DmMkNYO6Xb4k,5mjYQaktjmjcMKcUIcqz4s,None,2023-09-01,172965,None,0.628,0.523,11,-8.307,0,0.0946,0.70100,0.002740,0.2190,0.416,169.982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Closer,"The Chainsmokers, Halsey",Closer,0rSLgV8p5FzfnqlEk4GzxE,7BKLCZ1jbUBVqRi2FVlTVw,None,2016-07-29,244960,None,0.748,0.524,8,-5.599,1,0.0338,0.41400,0.000000,0.1110,0.661,95.010
3,Uptown Funk (feat. Bruno Mars),"Mark Ronson, Bruno Mars",Uptown Special,3vLaOYCNCzngDf8QdBg2V1,32OlwWuMpZ6b0aN2RZOeMS,None,2015-01-12,269667,None,0.856,0.609,0,-7.223,1,0.0824,0.00801,0.000082,0.0344,0.928,114.988
20,My House,Flo Rida,My House,5lkNnHVlnCCCV304t89wOH,6Knv6wdA0luoMUuuoYi2i1,None,2015-04-07,192191,None,0.688,0.702,7,-4.792,0,0.0499,0.02150,0.000000,0.1280,0.740,94.006
63,PILLOWTALK,ZAYN,Mind Of Mine (Deluxe Edition),5amj9zNeZ3B2EdpBgXrOZ0,0PDUDa38GO8lMxLCRc4lL1,None,2016-03-25,202747,None,0.584,0.700,11,-4.275,1,0.0456,0.11700,0.000000,0.0939,0.438,124.944
